In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Installation commands
!unzip '/content/drive/MyDrive/stanford-parser-full-2018-02-27.zip'
!pip install svgling

Archive:  /content/drive/MyDrive/stanford-parser-full-2018-02-27.zip
   creating: stanford-parser-full-2018-02-27/
  inflating: stanford-parser-full-2018-02-27/README.txt  
   creating: stanford-parser-full-2018-02-27/data/
  inflating: stanford-parser-full-2018-02-27/data/testsent.txt  
 extracting: stanford-parser-full-2018-02-27/data/chinese-onesent-unseg-gb18030.txt  
 extracting: stanford-parser-full-2018-02-27/data/english-onesent.txt  
 extracting: stanford-parser-full-2018-02-27/data/chinese-onesent-gb18030.txt  
 extracting: stanford-parser-full-2018-02-27/data/chinese-onesent-utf8.txt  
  inflating: stanford-parser-full-2018-02-27/data/french-onesent.txt  
 extracting: stanford-parser-full-2018-02-27/data/arabic-onesent-utf8.txt  
  inflating: stanford-parser-full-2018-02-27/data/chinese-onesent-unseg-utf8.txt  
 extracting: stanford-parser-full-2018-02-27/data/german-onesent.txt  
  inflating: stanford-parser-full-2018-02-27/data/pos-sentences.txt  
 extracting: stanford-par

In [ ]:
#Import statements

import os
from nltk.parse.stanford import StanfordParser
from nltk import ParentedTree, Tree
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
#Setting java path
java_path = "/usr/lib/jvm/java-11-openjdk-amd64/bin/java"
os.environ['JAVAHOME'] = java_path

In [ ]:
sp = StanfordParser(path_to_jar='stanford-parser-full-2018-02-27/stanford-parser.jar',
                    path_to_models_jar='stanford-parser-full-2018-02-27/stanford-parser-3.9.1-models.jar')

<ipython-input-6-36d49bdf4ce2>:1: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  sp = StanfordParser(path_to_jar='stanford-parser-full-2018-02-27/stanford-parser.jar',


In [ ]:
stopwords_set = set(['a', 'an', 'the', 'is','to','The','in','of','us'])

In [ ]:
def convert_isl(parsetree):
  dict = {}
  parenttree= ParentedTree.convert(parsetree)
  for sub in parenttree.subtrees():
    dict[sub.treeposition()]=0
  #"----------------------------------------------"
  isltree=Tree('ROOT',[])
  i=0
  for sub in parenttree.subtrees():
    if(sub.label()=="NP" and dict[sub.treeposition()]==0 and dict[sub.parent().treeposition()]==0):
      dict[sub.treeposition()]=1
      isltree.insert(i,sub)
      i=i+1

    if(sub.label()=="VP" or sub.label()=="PRP"):
      for sub2 in sub.subtrees():
        if((sub2.label()=="NP" or sub2.label()=='PRP') and dict[sub2.treeposition()]==0 and dict[sub2.parent().treeposition()]==0):
          dict[sub2.treeposition()]=1
          isltree.insert(i,sub2)
          i=i+1

  for sub in parenttree.subtrees():
    for sub2 in sub.subtrees():
      if(len(sub2.leaves())==1 and dict[sub2.treeposition()]==0 and dict[sub2.parent().treeposition()]==0):
          dict[sub2.treeposition()]=1
          isltree.insert(i,sub2)
          i=i+1

  return isltree

In [ ]:
def text_to_isl(sentence):
  pattern = r'[^\w\s]'

# Remove the punctuation marks from the sentence
  sentence = re.sub(pattern, '', sentence)
  englishtree=[tree for tree in sp.parse(sentence.split())]
  parsetree=englishtree[0]
  #print(parsetree)
  isl_tree = convert_isl(parsetree)
  words=parsetree.leaves()
  lemmatizer = WordNetLemmatizer()
  ps = PorterStemmer()
  lemmatized_words=[]
  for w in words:
    #w = ps.stem(w)
   lemmatized_words.append(lemmatizer.lemmatize(w))
   islsentence = ""
   for w in lemmatized_words:
    if w not in stopwords_set:
      islsentence+=w
      islsentence+=" "
  #islsentence = clean(words)
  islsentence = islsentence.lower()
  isltree=[tree for tree in sp.parse(islsentence.split())]
  return islsentence

In [ ]:
!pip install moviepy
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.5 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube
def get_yt(link,path):
  yt = YouTube(link)
  yt.streams.filter(file_extension="mp4").get_by_resolution("360p").download(path)

In [ ]:
from moviepy.editor import *
import os

def cut_vid(filename,yt_name,start_min,start_sec,end_min,end_sec):
  clip = VideoFileClip(os.path.join(yt_path,yt_name+'.mp4'))
  clip1 = clip.subclip((start_min,start_sec),(end_min,end_sec))
  clip1.write_videofile(os.path.join('/content/NLP_dataset',filename+'.mp4'),codec='libx264')

In [ ]:
!mkdir NLP_dataset
!mkdir yt

In [ ]:
root_path = '/content/NLP_dataset'
yt_path = '/content/yt'

In [ ]:
import pandas as pd

In [ ]:
NLP_videos = pd.read_csv('/content/drive/MyDrive/NLP_videos.csv')

In [ ]:
import pandas as pd
from IPython.display import HTML
from base64 import b64encode

In [ ]:
NLP_videos = pd.read_csv('/content/drive/MyDrive/NLP_videos.csv')
NLP_videos.head()

,Name,yt_name,Link,start_min,start_sec,end_min,end_sec
0,she,she,https://youtu.be/BXw0xmFB8pQ,0,0,0,3
1,eat,eat,https://youtu.be/d4nkMcBvUZM,0,0,0,3
2,time,time,https://youtu.be/4K-4FU5l7cY,0,0,0,3
3,day,day,https://youtu.be/11derBixy2I,0,0,0,2
4,her,her,https://youtu.be/qBNL8tfKk_8,0,0,0,2


In [ ]:
def text_to_vid(input_text):
  NLP_videos = pd.read_csv('/content/drive/MyDrive/NLP_videos.csv')
  root_path = '/content/NLP_dataset'
  yt_path = '/content/yt'
  videos = []
  clips=[]
  sentence = text_to_isl(input_text)
  print(sentence)
  words = sentence.split()
  for i in words:
    if (NLP_videos['Name'].eq(i)).any():
      idx = NLP_videos.index[NLP_videos['Name'] == i].tolist()
      get_yt(NLP_videos['Link'].iloc[idx[0]],yt_path)
      cut_vid(i,NLP_videos['yt_name'].iloc[idx[0]],NLP_videos['start_min'].iloc[idx[0]],NLP_videos['start_sec'].iloc[idx[0]],NLP_videos['end_min'].iloc[idx[0]],NLP_videos['end_sec'].iloc[idx[0]])
      videos.append(os.path.join(root_path,i+'.mp4'))
    else:
      for letter in i:
        idx = NLP_videos.index[NLP_videos['Name'] == letter].tolist()
        #print(letter,idx)
        get_yt(NLP_videos['Link'].iloc[idx[0]],yt_path)
        cut_vid(letter,NLP_videos['yt_name'].iloc[idx[0]],NLP_videos['start_min'].iloc[idx[0]],NLP_videos['start_sec'].iloc[idx[0]],NLP_videos['end_min'].iloc[idx[0]],NLP_videos['end_sec'].iloc[idx[0]])
        videos.append(os.path.join(root_path,letter+'.mp4'))
  for i in videos:
    clip = VideoFileClip(i)
    clips.append(clip)

  final = concatenate_videoclips(clips, method="compose")
  final.write_videofile("merged.mp4")

# Speech To Text

In [ ]:
!pip install SpeechRecognition
!pip install pyaudio
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 43.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio, which is required to install pyproject.toml-based projects
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pub

In [ ]:
import speech_recognition as sr
r = sr.Recognizer()

In [ ]:
AUDIO_FILE = ("/content/NLP_test (2).wav")

with sr.AudioFile(AUDIO_FILE) as source:
    #reads the audio file. Here we use record instead of
    #listen
    audio = r.record(source)
speech=r.recognize_google(audio)

In [ ]:
print(speech)

I go to school on Saturday


# Speech to Video

In [ ]:
text_to_vid(speech)

i go school on saturday 
Moviepy - Building video /content/NLP_dataset/i.mp4.
MoviePy - Writing audio in iTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/NLP_dataset/i.mp4



Moviepy - Done !
Moviepy - video ready /content/NLP_dataset/i.mp4
Moviepy - Building video /content/NLP_dataset/go.mp4.
MoviePy - Writing audio in goTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/NLP_dataset/go.mp4



Moviepy - Done !
Moviepy - video ready /content/NLP_dataset/go.mp4
Moviepy - Building video /content/NLP_dataset/school.mp4.
MoviePy - Writing audio in schoolTEMP_MPY_wvf_snd.mp3


...

MoviePy - Done.
Moviepy - Writing video /content/NLP_dataset/saturday.mp4



Moviepy - Done !
Moviepy - video ready /content/NLP_dataset/saturday.mp4
Moviepy - Building video merged.mp4.
MoviePy - Writing audio in mergedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video merged.mp4



t:  15%|█▍        | 45/306 [00:00<00:01, 224.44it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.9/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/NLP_dataset/i.mp4, 518400 bytes wanted but 0 bytes read,at frame 50/52, at time 2.00/2.04 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  39%|███▉      | 120/306 [00:01<00:02, 89.95it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.9/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/NLP_dataset/go.mp4, 518400 bytes wanted but 0 bytes read,at frame 50/52, at time 2.00/2.04 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  60%|█████▉    | 183/306 [00:01<00:01, 119.75it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.9/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/NLP_dataset/school.mp4,

Moviepy - Done !
Moviepy - video ready merged.mp4


In [ ]:
mp4 = open('merged.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
  <video width=400 controls>
      <source src="%s" type="video/mp4">
  </video>
  """ % data_url)